<a href="https://colab.research.google.com/github/songseunghei/DeepLearning/blob/main/VGG16_CAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2 as cv
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from google.colab.patches import cv2_imshow
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input,decode_predictions


from google.colab import drive
drive.mount('/content/drive')

model = VGG16(weights='imagenet')
img = cv.imread('cat.jpg')

img = cv.imread('/content/sample_data/cat.jpg')

x=np.reshape(cv.resize(img,(224,224)),(1,224,224,3))
x=preprocess_input(x)
preds = model.predict(x)
top5=decode_predictions(preds,top=5)[0]
print('예측결과:',top5)

for i in range(5):
  cv.putText(img,top5[i][1] + ':' + str(top5[i][2]),(10,20+i*20), cv.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1)
cv2_imshow(img)

cv.waitKey()
cv.destroyAllWindows()

model.summary()
np.argmax(preds[0])

from keras import backend as K
#wood_rabbit_output =model.output[:,330]
target_output = model.output[:, 330]
#last_conv_layer_name="conv5_block3_out"
last_conv_layer_name = "block5_conv3"
last_conv_layer =model.get_layer(last_conv_layer_name)

grads = K.gradients(target_output, last_conv_layer.output)[0]

pooled_grads = K.mean(grads, axis=(0,1,2))
iterate =K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value=iterate([x])

for i in range(512):
  conv_layer_output_value[:,:,i] *= pooled_grads_value[i]
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap,0)
heatmap /= np.max(heatmap)
img = cv.imread('/content/sample_data/cat.jpg')
heatmap = cv.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap=np.uint8(255*heatmap)
heatmap=cv.applyColorMap(heatmap,cv.COLORMAP_JET)
superimposed_img = heatmap * 0.4 +img
cv2_imshow(superimposed_img)

KeyboardInterrupt: ignored